# Literal Listener

## Set-up

See [colors_overview.ipynb](colors_overview.ipynb) for set-up in instructions and other background details.

In [1]:
from colors import ColorsCorpusReader
import os
from sklearn.model_selection import train_test_split
from torch_color_selector import (
    ColorizedNeuralListener, create_example_dataset)
import utils
from utils import START_SYMBOL, END_SYMBOL, UNK_SYMBOL
import numpy as np

In [2]:
utils.fix_random_seeds()

In [3]:
COLORS_SRC_FILENAME = os.path.join(
    "data", "colors", "filteredCorpus.csv")

## Dataset

In [4]:
# This variable will toggle whether we're training the listener or the listener-hallucinating speaker
agent = 'speaker'

In [5]:
def load_from_pickle():
    import pickle 
    with open('dev_vocab_'+agent+'.pickle', 'rb') as handle:
        dev_vocab = pickle.load(handle)
    with open('dev_seqs_test.pickle', 'rb') as handle:
        dev_seqs_test = pickle.load(handle)
    with open('dev_seqs_train_'+agent+'.pickle', 'rb') as handle:
        dev_seqs_train = pickle.load(handle)
    with open('dev_cols_test.pickle', 'rb') as handle:
        dev_cols_test = pickle.load(handle)
    with open('dev_cols_train_'+agent+'.pickle', 'rb') as handle:
        dev_cols_train = pickle.load(handle)
    with open('embedding.pickle', 'rb') as handle:
        embedding = pickle.load(handle)
    return dev_vocab, dev_seqs_test, dev_seqs_train, dev_cols_test, dev_cols_train, embedding

dev_vocab, dev_seqs_test, dev_seqs_train, dev_cols_test, dev_cols_train, embedding = load_from_pickle()

## Initial model

The first model is configured right now to be a small model run for just a few iterations. It should be enough to get traction, but it's unlikely to be a great model. You are free to modify this configuration if you wish; it is here just for demonstration and testing:

In [6]:
dev_mod = ColorizedNeuralListener(
    dev_vocab, 
    embed_dim=10, 
    hidden_dim=10, 
    max_iter=5, 
    batch_size=128)

Using cuda


In [7]:
#_ = dev_mod.fit(dev_cols_train, dev_seqs_train)

We can also see the model's predicted sequences given color context inputs:

In [8]:
#dev_mod.predict(dev_cols_test[:1], dev_seqs_train[:1])

As discussed in [colors_overview.ipynb](colors_overview.ipynb), our primary metric is `listener_accuracy`:

In [9]:
#dev_mod.listener_accuracy(dev_cols_test, dev_seqs_test)

In [10]:
#dev_seqs_train[:1]

## Generate Embeddings

In [11]:
embedding = np.random.normal(
            loc=0, scale=0.01, size=(len(dev_vocab), 100))

## Literal Listener

Let's start with the toy dataset.

In [12]:
toy_color_seqs, toy_word_seqs, toy_vocab = create_example_dataset(
    group_size=50, vec_dim=2)

In [13]:
toy_color_seqs_train, toy_color_seqs_test, toy_word_seqs_train, toy_word_seqs_test = \
    train_test_split(toy_color_seqs, toy_word_seqs)

In [14]:
toy_mod = ColorizedNeuralListener(
    toy_vocab, 
    embed_dim=100, 
    embedding=embedding,
    hidden_dim=100, 
    max_iter=100, 
    batch_size=128)

Using cuda


In [15]:
_ = toy_mod.fit(toy_color_seqs_train, toy_word_seqs_train)

ColorizedNeuralListenerEncoder cpu
ColorizedNeuralListenerEncoderDecoder cpu
Train: Epoch 1; err = 1.0985796451568604; time = 2.127473831176758
Train: Epoch 2; err = 1.0909310579299927; time = 0.02800583839416504
Train: Epoch 3; err = 1.0653038024902344; time = 0.027005910873413086
Train: Epoch 4; err = 1.0412218570709229; time = 0.02700638771057129
Train: Epoch 5; err = 1.0006825923919678; time = 0.027006149291992188
Train: Epoch 6; err = 0.9714384078979492; time = 0.027005672454833984
Train: Epoch 7; err = 0.9277397990226746; time = 0.02700638771057129
Train: Epoch 8; err = 0.8961005210876465; time = 0.02800607681274414
Train: Epoch 9; err = 0.8725563883781433; time = 0.025005817413330078
Train: Epoch 10; err = 0.8725194334983826; time = 0.02800583839416504
Train: Epoch 11; err = 0.8169789910316467; time = 0.02300572395324707
Train: Epoch 12; err = 0.9166122078895569; time = 0.025005578994750977
Train: Epoch 13; err = 0.8028489947319031; time = 0.02400493621826172
Train: Epoch 14; er

In [16]:
preds = toy_mod.predict(toy_color_seqs_test, toy_word_seqs_test)
correct = sum([1 if x == 2 else 0 for x in preds])
print(correct, "/", len(preds), correct/len(preds))

11 / 38 0.2894736842105263


If that worked, then you can now try this model on SCC problems!

In [17]:
dev_color_mod = ColorizedNeuralListener(
    dev_vocab,
    embed_dim=100,
    embedding=embedding,
    hidden_dim=100, 
    max_iter=100,
    batch_size=32,
    dropout_prob=0.,
    eta=0.001,
    lr_rate=0.96,
    warm_start=True,
    device='cuda')
# Uncomment line if you want to continue training the previous model
# literal_listener.load_model("literal_listener.pt")


Using cuda


In [18]:
_ = dev_color_mod.fit(dev_cols_train, dev_seqs_train)

ColorizedNeuralListenerEncoder cuda
ColorizedNeuralListenerEncoderDecoder cuda


C:\Users\Chris\Github\cs224u\torch_color_selector.py:77: ComplexWarning: Casting complex values to real discards the imaginary part
  color_seqs = torch.FloatTensor(color_seqs)


Train: Epoch 1; err = 581.3406069278717; time = 15.014360427856445
Train: Epoch 2; err = 533.6950891017914; time = 14.874318361282349
Train: Epoch 3; err = 517.7756766676903; time = 15.107369661331177
Train: Epoch 4; err = 507.24547374248505; time = 14.967339277267456
Train: Epoch 5; err = 499.4843147993088; time = 14.547255277633667
Train: Epoch 6; err = 496.27277106046677; time = 14.581241846084595
Train: Epoch 7; err = 490.329795897007; time = 14.60525894165039
Train: Epoch 8; err = 497.0830498933792; time = 14.471238613128662
Train: Epoch 9; err = 484.0396258831024; time = 15.058359146118164
Train: Epoch 10; err = 482.9984503388405; time = 14.741277933120728
Train: Epoch 11; err = 477.67922580242157; time = 14.498224020004272
Train: Epoch 12; err = 475.7627447247505; time = 14.43122911453247
Train: Epoch 13; err = 483.3060376048088; time = 14.620261669158936
Train: Epoch 14; err = 473.72228223085403; time = 14.64025616645813
0.00096
tensor([0.1595, 0.1872, 0.6532], device='cuda:0',

In [19]:
test_preds = dev_color_mod.predict(dev_cols_test, dev_seqs_test)
#dev_color_mod.predict(dev_cols_test, dev_seqs_test, probabilities=True)
train_preds = dev_color_mod.predict(dev_cols_train, dev_seqs_train)
#dev_color_mod.predict(dev_cols_test, dev_seqs_test, probabilities=True)

In [20]:
correct = sum([1 if x == 2 else 0 for x in test_preds])
print("test", correct, "/", len(test_preds), correct/len(test_preds))
correct = sum([1 if x == 2 else 0 for x in train_preds])
print("train", correct, "/", len(train_preds), correct/len(train_preds))

test 8710 / 11749 0.7413396884841263
train 14482 / 17623 0.8217670090223004


In [21]:
dev_color_mod.save_model("literal_listener_"+agent+"_split.pt")